Import the necessary libraries. Pandas is used to manipulate the data to be able to plot it onto a map using Geopandas and Plotly.

In [ ]:
import pandas as pd
import geopandas
import json
import plotly.express as px

Contract data may be obtained at the "Proactive Publication - Contracts over $10,000" portal located on the open.canada.ca website: https://open.canada.ca/data/en/dataset/d8f85d91-7dec-4fd1-8055-483b77225d8b/resource/fac950c0-00d5-4ec1-a4d3-9cbebf98a305

For the purpose of rendering the map, only the "contract_value" and "vendor_postal_code" attributes are necessary. You may choose to reduce the size of the CSV you retrieve from the portal by only selecting those attributes. Otherwise, you may keep them if you are interested in looking at them.

To retrieve government contract data from a specific fiscal year, click "Add Filter" on the website and select "reporting-period." From here, you are able to narrow it down to a specific fiscal year, e.g. 2006-2007-Q1, 2006-2007-Q2, and so on.

Load the CSV file and check the contents.

In [ ]:
contracts = pd.read_csv("realdata.csv")

In [ ]:
contracts

Next, the data is used to create a dataframe calculating the total dollar value sum of all the contracts awarded to companies based in that postal code. 

In [ ]:
total_contract_value = contracts.groupby('vendor_postal_code')['contract_value'].sum()
total_contract_value = total_contract_value.to_frame()
total_contract_value = total_contract_value.rename_axis('CFSAUID').reset_index()
total_contract_value


Unused "contract_discrepancy" dataframe. This is calculated by calcualating the discrepancy between the final total recorded dollar value of the contracts compared to the original total recorded dollar value of the contracts.

In [ ]:
contract_discrepancy = contracts.groupby('vendor_postal_code')['contract_value'].sum() - contracts.groupby('vendor_postal_code')['original_value'].sum()
contract_discrepancy

Read the shape file containing the Postal Code boundaries. We then narrow it down to the most essential parts: The postal code itself, the Province ID code, and the actual geometry/borders.

In [ ]:
postal_codes = geopandas.read_file("lfsa000b21a_e\lfsa000b21a_e.shp")
postal_codes = postal_codes[['CFSAUID', 'PRUID', 'geometry']]
print(type(postal_codes))

In [ ]:
postal_codes['contract_value'] = pd.Series(dtype=int)


In [ ]:
new_pc = postal_codes.merge(total_contract_value, on="CFSAUID")
new_pc
new_pc = new_pc.drop("contract_value_x", axis=1)


In [ ]:
new_pc = new_pc.rename(columns={"contract_value_y": "contract_value"})
new_pc

In [ ]:
bc_pc = new_pc[new_pc.PRUID == "59"]
bc_pc


There is probably a more efficient way to do this, but once you have narrowed the dataframe into the province the location you are trying to graph, delete rows of postal codes from the beginning and the end until you have all the postal codes.

In [ ]:
# Define the number of rows to delete from the beginning and the end
num_to_delete_from_beginning = 97
num_to_delete_from_end = 57

# Select the rows to keep (excluding the first 'num_to_delete_from_beginning' and last 'num_to_delete_from_end' rows)
rows_to_keep = slice(num_to_delete_from_beginning, len(bc_pc) - num_to_delete_from_end)

# Create a new dataframe containing the desired rows
bc_pc1 = bc_pc.iloc[rows_to_keep]

bc_pc1


In [ ]:
with open ("geojsons\\vancouverfixedFINAL.geojson", "r") as infile:
    vancouver = json.load(infile)

In [ ]:
map = px.choropleth_mapbox(bc_pc1, geojson=vancouver, 
                           locations="CFSAUID", featureidkey="properties.CFSAUID",color="contract_value", range_color=(bc_pc1['contract_value'].min(), bc_pc1['contract_value'].max()), mapbox_style="carto-positron",
                           zoom=7, center = {"lat": 49.280000, "lon": -123.120405},
                           opacity=0.5)


In [ ]:
map.show()

In [ ]:
map.write_html("vancouver_map.html")

A little unrelated snippet of code that allows you to look at what the largest contracts awarded in that postal code are.

In [ ]:
contracts[contracts['vendor_postal_code'] == "M4Y"].nlargest(10, "contract_value")